In [5]:
import pandas as pd
from PIL import Image

In [2]:
import torch
from transformers import CLIPModel, CLIPProcessor

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [7]:
data_path = '../data/'

df = pd.read_csv(data_path + 'data.csv')
df.head()

,filename,label
0,00_DSCN9645_2,3
1,00_20d5f703-IMG_20190430_213225,1
2,109_DSCN9487,5
3,137_DSCN0889,3
4,46_DSCN9587,3


In [9]:
img = Image.open(data_path + 'images/' + df['filename'].iloc[0] + '.jpg')

In [11]:
inputs = processor(text=["a photo of a cat", "a photo of a man"], images=img, return_tensors="pt", padding=True)

In [12]:
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image

In [16]:
probs = logits_per_image.softmax(dim=1)
probs

tensor([[0.2628, 0.7372]], grad_fn=<SoftmaxBackward0>)

In [18]:
import json
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import clip
from transformers import CLIPProcessor, CLIPModel

import tqdm

In [19]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/home/kcdh/miniconda3/envs/hanish/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
class image_title_dataset():
    def __init__(self, list_image_path,list_txt):
        # Initialize image paths and corresponding texts
        self.image_path = list_image_path
        # Tokenize text using CLIP's tokenizer
        self.title  = clip.tokenize(list_txt)

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        # Preprocess image using CLIP's preprocessing function
        # image = preprocess(Image.open(self.image_path[idx])) # original
        image = clip.preprocess(Image.open(self.image_path[idx]))
        title = self.title[idx]
        return image, title

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

num_epochs = 30
for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()

        images,texts = batch 
        
        images= images.to(device)
        texts = texts.to(device)

        # Forward pass
        logits_per_image, logits_per_text = model(images, texts)

        # Compute loss
        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

        # Backward pass
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)

        pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")